In [2]:
# importing the example model and the sparseQFCA module
using sparseQFCA, JSON, SparseArrays, DelimitedFiles
@time open("iMM1415.json") do f
    dict = JSON.parse(f);
    data = dict["reactions"];
    global name = [(data[i])["name"] for i in 1:length(data)];
    global lower_bound = [(data[i])["lower_bound"] for i in 1:length(data)];
    global upper_bound = [(data[i])["upper_bound"] for i in 1:length(data)];
    global gene_reaction_rule = [(data[i])["gene_reaction_rule"] for i in 1:length(data)];
    metabolites = [(data[i])["metabolites"] for i in 1:length(data)];
    id = [((dict["metabolites"])[i])["id"] for i in 1:length(dict["metabolites"])];
    S = Matrix{Float64}(undef, length(id), length(metabolites));
    for i in 1:length(metabolites)
        for j in 1:length(id)
            S[j, i] = get(metabolites[i], id[j], 0)
        end
    end
    global S = sparse(S);
end
@assert typeof(S) == SparseMatrixCSC{Float64,Int64}
rev = readdlm("rev.csv", header = false)[:, 1] .== 1
@assert typeof(rev) == BitArray{1}
# finding all the flux coupling relations among the reactions
certificates, blocked, fctable = @time QFCA(S, rev)
println("The answer is $(all((readdlm("blocked.csv", header = false) .== 1) .== blocked) ? "correct" : "wrong").")
println("The answer is $(all(readdlm("fctable.csv", header = false) .== fctable) ? "correct" : "wrong").")
open("certificates.txt", "w") do io
    writedlm(io, certificates)
end

  2.841473 seconds (29.15 M allocations: 760.187 MiB, 13.26% gc time, 23.18% compilation time)
861.983010 seconds (3.91 G allocations: 351.983 GiB, 7.58% gc time, 1.66% compilation time)
The answer is correct.
The answer is correct.
